In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector
import json

In [2]:
from spotify_api import authentication, get_track_features, get_track, get_artist

In [ ]:
authentication()

In [3]:
#connect mysql database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="210502",
    database="timeless_song"
)
cursor = conn.cursor()


In [15]:
cursor.execute("SELECT * FROM top_80s")
top_80s = cursor.fetchall()
top_80s = pd.DataFrame(top_80s, columns=cursor.column_names)
top_80s.drop(columns=['target'], inplace=True)
top_80s['Song'] = top_80s['Song'].str.replace('\"', '')
top_80s['Song'] = top_80s['Song'].str.upper()
top_80s = top_80s.rename(columns={'Song': 'song'})
top_80s.rename(columns={'Artist Names': 'artist'}, inplace=True)
top_80s['artist'] = top_80s['artist'].str.replace('\"', '')
top_80s['artist'] = top_80s['artist'].str.upper()
top_80s['song_id'] = top_80s['uri'].str.split(':').str[-1]
top_80s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,country,song_id
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,0.000050,0.3130,0.904,123.815,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,0.000000,0.0959,0.596,116.624,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,0.000248,0.1380,0.237,93.633,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,0.000311,0.0873,0.548,108.873,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,0.002940,0.0714,0.740,117.401,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,0.000011,0.1580,0.267,130.465,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,0.000002,0.4850,0.963,149.206,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,0.000002,0.6310,0.813,105.390,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,0.003360,0.0884,0.889,176.737,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU


In [16]:
top_80s['genres'] = None
for index, row in top_80s.iterrows():
    song_id = row['song_id']
    track = get_track(song_id)
    artist_ids = [artist['id'] for artist in track['artists']]
    genres = []
    for artist_id in artist_ids:
        artist_info = get_artist(artist_id)
        genres.extend(artist_info.get('genres', []))
        genres = list(set(genres))
    top_80s.at[index, 'genres'] = genres
top_80s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,country,song_id,genres
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,0.3130,0.904,123.815,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,"[mellow gold, disco, soft rock, australian dan..."
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,0.0959,0.596,116.624,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,"[new romantic, soft rock, new wave pop]"
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,0.1380,0.237,93.633,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,"[disco, quiet storm, soul, soft rock, adult st..."
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,0.0873,0.548,108.873,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,"[glam metal, album rock, hard rock, soft rock]"
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,0.0714,0.740,117.401,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,"[permanent wave, rock, classic rock, album rock]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,0.1580,0.267,130.465,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,"[folk, singer-songwriter, mellow gold, heartla..."
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,0.4850,0.963,149.206,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,"[singer-songwriter, mellow gold, folk rock, al..."
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,0.6310,0.813,105.390,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,"[country, country dawn, classic country pop]"
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,0.0884,0.889,176.737,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,"[neue deutsche welle, austropop]"


In [102]:
# make genre column to have value that is common e.g. when there are soft rock, and classic rock => rock
# many kind of pop => pop
# combine each list of common genre to 1 genre
def combine_genre(genres):
    if 'soft rock' in genres:
        return 'rock'
    if 'classic rock' in genres:
        return 'rock'
    if 'new wave' in genres:
        return 'rock'
    if 'dance rock' in genres:
        return 'rock'
    if 'hard rock' in genres:
        return 'rock'
    if 'pop' in genres:
        return 'pop'
    if 'tropical' in genres:
        return 'pop'
    if 'dance pop' in genres:
        return 'pop'
    if 'new wave pop' in genres:
        return 'pop'
    if 'freestyle' in genres:
        return 'pop'
    if 'boy band' in genres:
        return 'pop'
    if 'christian music' in genres:
        return 'pop'
    if 'black comedy' in genres:
        return 'pop'
    if 'austropop' in genres:
        return 'pop'
    if 'canadian pop' in genres:
        return 'pop'
    if 'synthpop' in genres:
        return 'pop'
    if 'rock' in genres:
        return 'rock'
    if 'roots rock' in genres:
        return 'rock'
    if 'classic canadian rock' in genres:
        return 'rock'
    if 'pop rock' in genres:
        return 'rock'
    if 'hip hop' in genres:
        return 'hip hop'
    if 'old school hip hop' in genres:
        return 'hip hop'
    if 'hip house' in genres:
        return 'hip hop'
    if 'rap' in genres:
        return 'hip hop'
    if 'r&b' in genres:
        return 'r&b'
    if 'soul' in genres:
        return 'r&b'
    if 'funk' in genres:
        return 'r&b'
    if 'contemporary r&b' in genres:
        return 'r&b'
    if 'jazz' in genres:
        return 'jazz'
    if 'blues' in genres:
        return 'jazz'
    if 'vocal jazz' in genres:
        return 'jazz'
    if 'country' in genres:
        return 'country'
    if 'folk' in genres:
        return 'country'
    if 'reggae' in genres:
        return 'reggae'
    if 'reggae fusion' in genres:
        return 'reggae' 
    if 'latin' in genres:
        return 'latin'
    if 'metal' in genres:
        return 'metal'
    if 'punk' in genres:
        return 'punk'
    if 'disco' in genres:
        return 'disco'
    if 'house' in genres:
        return 'house'
    if 'techno' in genres:
        return 'house'
    if 'edm' in genres:
        return 'house'
    if 'electronic' in genres:
        return 'house'
    if 'classical' in genres:
        return 'classical'
    if 'opera' in genres:
        return 'classical'
    return 'other'

In [ ]:
top_80s['genre'] = top_80s['genres'].apply(combine_genre)
top_80s

In [23]:
final_top_80s = top_80s
final_top_80s.drop(columns=['genres'], inplace=True)
final_top_80s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,0.3130,0.904,123.815,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,0.0959,0.596,116.624,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,0.1380,0.237,93.633,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,0.0873,0.548,108.873,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,0.0714,0.740,117.401,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,0.1580,0.267,130.465,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,0.4850,0.963,149.206,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,0.6310,0.813,105.390,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,0.0884,0.889,176.737,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop


In [22]:
top_80s.to_csv('top_80s.csv', index=False)

In [26]:
cursor.execute("SELECT * FROM stream_count")
stream_count = cursor.fetchall()
stream_count = pd.DataFrame(stream_count, columns=cursor.column_names)
stream_count.rename(columns={'title': 'song'}, inplace=True)
stream_count['year'] = stream_count['date'].str.split('-').str[0].astype(int)
stream_count = stream_count[['song', 'artist', 'year', 'streams', 'url']]
stream_count = stream_count.fillna(0)
stream_count

,song,artist,year,streams,url
0,Chantaje (feat. Maluma),Shakira,2017,253019.0,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
1,Vente Pa' Ca (feat. Maluma),Ricky Martin,2017,223988.0,https://open.spotify.com/track/7DM4BPaS7uofFul...
2,Reggaetón Lento (Bailemos),CNCO,2017,210943.0,https://open.spotify.com/track/3AEZUABDXNtecAO...
3,Safari,"J Balvin, Pharrell Williams, BIA, Sky",2017,173865.0,https://open.spotify.com/track/6rQSrBHf7HlZjtc...
4,Shaky Shaky,Daddy Yankee,2017,153956.0,https://open.spotify.com/track/58IL315gMSTD37D...
...,...,...,...,...,...
1979995,Roses (feat. ROZES),The Chainsmokers,2017,4602.0,https://open.spotify.com/track/6O6M7pJLABmfBRo...
1979996,Cool Girl,Tove Lo,2017,4602.0,https://open.spotify.com/track/5dKyZWlgjWw1oJg...
1979997,Uptown Funk (feat. Bruno Mars),Mark Ronson,2017,4504.0,https://open.spotify.com/track/32OlwWuMpZ6b0aN...
1979998,Not Going Home (feat. Gia Koka),"DVBBS, CMC$",2017,4486.0,https://open.spotify.com/track/042x9bsUBWW2fJg...


In [50]:
stream_count = pd.read_csv('charts.csv') 
stream_count.rename(columns={'title': 'song'}, inplace=True)
stream_count['year'] = stream_count['date'].str.split('-').str[0].astype(int)
stream_count = stream_count[['song', 'artist', 'year', 'streams', 'url']]
stream_count = stream_count.fillna(0)
stream_count

,song,artist,year,streams,url
0,Chantaje (feat. Maluma),Shakira,2017,253019.0,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
1,Vente Pa' Ca (feat. Maluma),Ricky Martin,2017,223988.0,https://open.spotify.com/track/7DM4BPaS7uofFul...
2,Reggaetón Lento (Bailemos),CNCO,2017,210943.0,https://open.spotify.com/track/3AEZUABDXNtecAO...
3,Safari,"J Balvin, Pharrell Williams, BIA, Sky",2017,173865.0,https://open.spotify.com/track/6rQSrBHf7HlZjtc...
4,Shaky Shaky,Daddy Yankee,2017,153956.0,https://open.spotify.com/track/58IL315gMSTD37D...
...,...,...,...,...,...
26173509,BYE,Jaden,2021,0.0,https://open.spotify.com/track/3OUyyDN7EZrL7i0...
26173510,Pillars,My Anh,2021,0.0,https://open.spotify.com/track/6eky30oFiQbHUAT...
26173511,Gái Độc Thân,Tlinh,2021,0.0,https://open.spotify.com/track/2klsSb2iTfgDh95...
26173512,Renegade (feat. Taylor Swift),Big Red Machine,2021,0.0,https://open.spotify.com/track/1aU1wpYBSpP0M6I...


In [51]:
streaming_2021 = stream_count[stream_count['year'] == 2021]

streaming_2021.loc[:,'song'] = streaming_2021.loc[:,'song'].str.replace('\"', '')
streaming_2021.loc[:,'song'] = streaming_2021.loc[:,'song'].str.upper()

streaming_2021.loc[:,'artist'] = streaming_2021.loc[:,'artist'].str.replace('\"', '')
streaming_2021.loc[:,'artist'] = streaming_2021.loc[:,'artist'].str.upper()
streaming_2021

,song,artist,year,streams,url
63293,STAY GOLD,BTS,2021,54393.0,https://open.spotify.com/track/3Ys2PYl1wyPKQIw...
69327,CLOSE EYES,DVRST,2021,0.0,https://open.spotify.com/track/3CLSHJv5aUROAN2...
71618,PARÍS,INGRATAX,2021,0.0,https://open.spotify.com/track/4bF2y8rR7GUjrd5...
71725,DUVET,BÔA,2021,0.0,https://open.spotify.com/track/7urtwZEtOypb3r8...
478448,POBLADO - REMIX,"J BALVIN, KAROL G, NICKY JAM, CRISSIN, TOTOY E...",2021,69591.0,https://open.spotify.com/track/2K1CQMoG2Dy5nme...
...,...,...,...,...,...
26173509,BYE,JADEN,2021,0.0,https://open.spotify.com/track/3OUyyDN7EZrL7i0...
26173510,PILLARS,MY ANH,2021,0.0,https://open.spotify.com/track/6eky30oFiQbHUAT...
26173511,GÁI ĐỘC THÂN,TLINH,2021,0.0,https://open.spotify.com/track/2klsSb2iTfgDh95...
26173512,RENEGADE (FEAT. TAYLOR SWIFT),BIG RED MACHINE,2021,0.0,https://open.spotify.com/track/1aU1wpYBSpP0M6I...


In [52]:
sum_streaming_2021 = streaming_2021.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2021

,song,artist,year,streams,url
0,!,"SAMEY, GLEB",2021,3581.0,https://open.spotify.com/track/1A05ibu1DXGIt0F...
1,!!!,"NNS, KKN, RYU",2021,0.0,https://open.spotify.com/track/50F2cUDhrqVGoei...
2,#1090 ~THOUSAND DREAMS~,松本孝弘,2021,0.0,https://open.spotify.com/track/0Bh4YOM4VCSY0BO...
3,"#1TAKE, PT. 2","P110, MIST",2021,0.0,https://open.spotify.com/track/24bNtsddXyIJwWK...
4,#AICHOAI,"FLO D, GIANG NGUYỄN",2021,111969.0,https://open.spotify.com/track/5xOkL4eAs735nm6...
...,...,...,...,...,...
44409,지나오다,NILO,2021,0.0,https://open.spotify.com/track/0keu6b483OyisjQ...
44410,친구,BTS,2021,356265.0,https://open.spotify.com/track/2dkFOWqyiVFS3xs...
44411,키스는 다음에,LOONA,2021,93903.0,https://open.spotify.com/track/3DGKXLtVjawKFwe...
44412,항상 (HANGSANG),"J-HOPE, SUPREME BOI",2021,0.0,https://open.spotify.com/track/4mYu3kfBCW6qiTD...


In [53]:
streaming_2020 = stream_count[stream_count['year'] == 2020]

streaming_2020.loc[:,'song'] = streaming_2020.loc[:,'song'].str.replace('\"', '')
streaming_2020.loc[:,'song'] = streaming_2020.loc[:,'song'].str.upper()

streaming_2020.loc[:,'artist'] = streaming_2020.loc[:,'artist'].str.replace('\"', '')
streaming_2020.loc[:,'artist'] = streaming_2020.loc[:,'artist'].str.upper()
streaming_2020

,song,artist,year,streams,url
19128,RIDE IT,REGARD,2020,1956.0,https://open.spotify.com/track/2tnVG71enUj33Ic...
19234,+LINDA,DALEX,2020,1953.0,https://open.spotify.com/track/5cBrOhKDyiJF9bP...
19287,WOAH,LIL BABY,2020,35211.0,https://open.spotify.com/track/02RCbjb9czvQKNG...
19390,25/8,BAD BUNNY,2020,31854.0,https://open.spotify.com/track/4NqQaovM14WR2hN...
19698,KEII,ANUEL AA,2020,16155.0,https://open.spotify.com/track/5eEQCfq7MXQr6aH...
...,...,...,...,...,...
21437959,MAD AT DISNEY,SALEM ILESE,2020,0.0,https://open.spotify.com/track/7aGyRfJWtLqgJaZ...
21437960,EM ĐÃ THƯƠNG NGƯỜI TA HƠN ANH,NOO PHƯỚC THỊNH,2020,0.0,https://open.spotify.com/track/39t3ly1KisXk4u5...
21437961,3 1 0 7,"W/N, DUONG, NÂU",2020,0.0,https://open.spotify.com/track/42pZsfo15M63tgC...
21437962,MISSING YOU,"PHUONG LY, TINLE",2020,0.0,https://open.spotify.com/track/1AAGvzJRWMTKx7P...


In [54]:
sum_streaming_2020 = streaming_2020.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2020

,song,artist,year,streams,url
0,!,"SAMEY, GLEB",2020,133718.0,https://open.spotify.com/track/1A05ibu1DXGIt0F...
1,#%_! DIG SELV,ARTIGEARDIT,2020,85639.0,https://open.spotify.com/track/6Js5Mxd61ze79K8...
2,#213,YL,2020,49321.0,https://open.spotify.com/track/1FdmvW7UjsG7Uiu...
3,#24,DOLU KADEHI TERS TUT,2020,649313.0,https://open.spotify.com/track/6ihcJlZFxGvqIq7...
4,#24,"PROFJAM, BENJI PRICE",2020,131173.0,https://open.spotify.com/track/0wlOyxuja1SEoqE...
...,...,...,...,...,...
47885,어머님이 누구니 (WHO'S YOUR MAMA?) (FEAT. JESSI),J.Y. PARK,2020,0.0,https://open.spotify.com/track/7Hjmz2XTMCuFvKB...
47886,우린 친구뿐일까,SONDIA,2020,6075.0,https://open.spotify.com/track/0w27Ok6tBj1QBNq...
47887,지코 아무노래,HAMAH MUSIC,2020,20783.0,https://open.spotify.com/track/7CAigOoiIydhI1G...
47888,하계,"MAGGIE, NYAN, BRILLIANT MUSIC",2020,0.0,https://open.spotify.com/track/6Wtyb11rKgjVyDY...


In [55]:
streaming_2019 = stream_count[stream_count['year'] == 2019]

streaming_2019.loc[:,'song'] = streaming_2019.loc[:,'song'].str.replace('\"', '')
streaming_2019.loc[:,'song'] = streaming_2019.loc[:,'song'].str.upper()

streaming_2019.loc[:,'artist'] = streaming_2019.loc[:,'artist'].str.replace('\"', '')
streaming_2019.loc[:,'artist'] = streaming_2019.loc[:,'artist'].str.upper()
streaming_2019

,song,artist,year,streams,url
62401,RIDE IT,REGARD,2019,0.0,https://open.spotify.com/track/2tnVG71enUj33Ic...
62557,RANSOM,LIL TECCA,2019,0.0,https://open.spotify.com/track/6EOKwHETwSkZ9gW...
62715,IO E LEI,COMA,2019,0.0,https://open.spotify.com/track/0a1LpYLrp1VNwEe...
63033,VOICE,TAEYEON,2019,0.0,https://open.spotify.com/track/1esVK0wgkpFJ31e...
63086,SIREN SONG,MARUV,2019,0.0,https://open.spotify.com/track/1RgSs1Jy3QHiacs...
...,...,...,...,...,...
19330997,GET YOU THE MOON (FEAT. SNØW),KINA,2019,0.0,https://open.spotify.com/track/4ZLzoOkj0MPWrTL...
19330998,TRAMPOLINE,"SHAED, ZAYN",2019,0.0,https://open.spotify.com/track/4HBvwhhYo7YiYXk...
19330999,BLUEMING,IU,2019,0.0,https://open.spotify.com/track/4Dr2hJ3EnVh2Aao...
19331000,MY OH MY (FEAT. DABABY),CAMILA CABELLO,2019,0.0,https://open.spotify.com/track/3yOlyBJuViE2YSG...


In [56]:
sum_streaming_2019 = streaming_2019.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2019

,song,artist,year,streams,url
0,!,"SAMEY, GLEB",2019,421700.0,https://open.spotify.com/track/1A05ibu1DXGIt0F...
1,!,TRIPPIE REDD,2019,5077885.0,https://open.spotify.com/track/5a1ofLoPiAn5xxf...
2,# ENJOY YOUR LIFE,BLOUSON CHIEMI,2019,0.0,https://open.spotify.com/track/66TvvimCAB7DB07...
3,#24,DOLU KADEHI TERS TUT,2019,90529.0,https://open.spotify.com/track/6ihcJlZFxGvqIq7...
4,#302,KEN HIRAI,2019,591135.0,https://open.spotify.com/track/6XfTow9VT9OhotJ...
...,...,...,...,...,...
44690,두근거려 (BEAUTIFUL),BAEKHYUN,2019,0.0,https://open.spotify.com/track/7rxnGhTlqU3FRUz...
44691,믿어줄래,GOT7,2019,57825.0,https://open.spotify.com/track/3t9aeFQEtd2dhuT...
44692,첫사랑,EPITONE PROJECT,2019,0.0,https://open.spotify.com/track/7EBnhmiVuuIz3p4...
44693,첫사랑,SONDIA,2019,0.0,https://open.spotify.com/track/6ofS1EG2nH01GL5...


In [57]:
streaming_2018 = stream_count[stream_count['year'] == 2018]

streaming_2018.loc[:,'song'] = streaming_2018.loc[:,'song'].str.replace('\"', '')
streaming_2018.loc[:,'song'] = streaming_2018.loc[:,'song'].str.upper()

streaming_2018.loc[:,'artist'] = streaming_2018.loc[:,'artist'].str.replace('\"', '')
streaming_2018.loc[:,'artist'] = streaming_2018.loc[:,'artist'].str.upper()
streaming_2018

,song,artist,year,streams,url
1264,MINE,BAZZI,2018,62549.0,https://open.spotify.com/track/6tHWl8ows5JOZq9...
2390,OUTCAST,FOOL,2018,15936.0,https://open.spotify.com/track/6jnDfM8vtRWgN7R...
2545,SKYLDIG,BLAK,2018,10817.0,https://open.spotify.com/track/4k9zexSmoSS6EER...
3069,ENKELIT,CHEEK,2018,18974.0,https://open.spotify.com/track/6hAKGURpSwL2tvm...
4328,44 MORE,LOGIC,2018,1367372.0,https://open.spotify.com/track/6iyp6udVpLuq3MC...
...,...,...,...,...,...
25276069,OJALÁ (FEAT. DARELL),"DE LA GHETTO, ALMIGHTY, BRYANT MYERS",2018,1178.0,https://open.spotify.com/track/3EMDvnVpQd9RZJv...
25276070,LO QUE PASA EN LA NOCHE,MANO ARRIBA,2018,1178.0,https://open.spotify.com/track/2eOleVJlGvBE027...
25276071,EL EQUIVOCADO,MANO ARRIBA,2018,1170.0,https://open.spotify.com/track/5vy1C7DD9xJ5fRB...
25276072,QUE FUI TU AMANTE,EL GUCCI Y SU BANDA,2018,1165.0,https://open.spotify.com/track/1fmiCxwEbZFIszI...


In [58]:
sum_streaming_2018 = streaming_2018.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2018

,song,artist,year,streams,url
0,!,O.S.T.R.,2018,123714.0,https://open.spotify.com/track/5dwTUHjTcaMMkCD...
1,!I'LL BE BACK!,RILÈS,2018,36343.0,https://open.spotify.com/track/0fROT4kK5oTm8xO...
2,#AGORA ACABOU,"MC HENRY P, MC DANILO",2018,0.0,https://open.spotify.com/track/0cXSaSdx6Bhf6cA...
3,#B3,RK,2018,127295.0,https://open.spotify.com/track/7ivouaHYYikXDeh...
4,#B5,RK,2018,0.0,https://open.spotify.com/track/6pL93KyT7UhC48s...
...,...,...,...,...,...
48446,항상 (HANGSANG),"J-HOPE, SUPREME BOI",2018,141250.0,https://open.spotify.com/track/4mYu3kfBCW6qiTD...
48447,혼자 NOBODY KNOWS,GOT7,2018,6416.0,https://open.spotify.com/track/7HZsEJcqeDKTgzH...
48448,혼잣말 TALKING TO,DAY6,2018,0.0,https://open.spotify.com/track/5DzVHa9drSwpBRL...
48449,환상통 ONLY ONE YOU NEED,JONGHYUN,2018,0.0,https://open.spotify.com/track/51Men21ginqHFcS...


In [59]:
streaming_2017 = stream_count[stream_count['year'] == 2017]

streaming_2017.loc[:,'song'] = streaming_2017.loc[:,'song'].str.replace('\"', '')
streaming_2017.loc[:,'song'] = streaming_2017.loc[:,'song'].str.upper()

streaming_2017.loc[:,'artist'] = streaming_2017.loc[:,'artist'].str.replace('\"', '')
streaming_2017.loc[:,'artist'] = streaming_2017.loc[:,'artist'].str.upper()
streaming_2017

,song,artist,year,streams,url
0,CHANTAJE (FEAT. MALUMA),SHAKIRA,2017,253019.0,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
1,VENTE PA' CA (FEAT. MALUMA),RICKY MARTIN,2017,223988.0,https://open.spotify.com/track/7DM4BPaS7uofFul...
2,REGGAETÓN LENTO (BAILEMOS),CNCO,2017,210943.0,https://open.spotify.com/track/3AEZUABDXNtecAO...
3,SAFARI,"J BALVIN, PHARRELL WILLIAMS, BIA, SKY",2017,173865.0,https://open.spotify.com/track/6rQSrBHf7HlZjtc...
4,SHAKY SHAKY,DADDY YANKEE,2017,153956.0,https://open.spotify.com/track/58IL315gMSTD37D...
...,...,...,...,...,...
25992090,TWO GHOSTS,HARRY STYLES,2017,0.0,https://open.spotify.com/track/4B1rpPmQXwj78wk...
25992091,APOCALIPSIS ZOMBI,EL CUARTETO DE NOS,2017,0.0,https://open.spotify.com/track/063bMRonMLP3UEZ...
25992092,HEY THERE DELILAH,PLAIN WHITE T'S,2017,0.0,https://open.spotify.com/track/5O3eNPhckrOCxIB...
25992093,ME REHÚSO,DANNY OCEAN,2017,0.0,https://open.spotify.com/track/6Zosz8w6CmdGMZM...


In [60]:
sum_streaming_2017 = streaming_2017.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2017

,song,artist,year,streams,url
0,#0000FF,JASMINE SOKKO,2017,0.0,https://open.spotify.com/track/6dAROoyDvypjygW...
1,#23,DOLU KADEHI TERS TUT,2017,0.0,https://open.spotify.com/track/69LV8HvCI5vAMpB...
2,#9 - LIVE FROM HIGH SIERRA MUSIC FESTIVAL 2014,MOON HOOCH,2017,0.0,https://open.spotify.com/track/4eAJM4eDq4UYyDT...
3,#99,JVG,2017,31826.0,https://open.spotify.com/track/69lSnKz6dwvIxJ9...
4,#ASKIP,BLACK M,2017,296862.0,https://open.spotify.com/track/4T7fVWFjYNwkuCK...
...,...,...,...,...,...
55299,"태양의 후예 (ORIGINAL TELEVISION SOUNDTRACK), PT. 6...",K.WILL,2017,0.0,https://open.spotify.com/track/5bwZjteR12LZdlW...
55300,피 땀 눈물,BTS,2017,70997.0,https://open.spotify.com/track/1DyOhkfykPENmHb...
55301,함께 CURE,"TAEYONG, YOO YOUNG JIN",2017,0.0,https://open.spotify.com/track/2OTSH26Ohg9GPRX...
55302,혼자 하는 사랑 LONELY LOVE,SEOHYUN,2017,0.0,https://open.spotify.com/track/7d1BFNh1Cpg98vp...


In [39]:
final_top_80s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,0.3130,0.904,123.815,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,0.0959,0.596,116.624,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,0.1380,0.237,93.633,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,0.0873,0.548,108.873,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,0.0714,0.740,117.401,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,0.1580,0.267,130.465,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,0.4850,0.963,149.206,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,0.6310,0.813,105.390,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,0.0884,0.889,176.737,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop


In [68]:
merged_df = pd.merge(final_top_80s, sum_streaming_2021, on=['song', 'artist'], how='left')
merged_df = merged_df.sort_values(by='Hot100 Rank')
merged_df

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre,year,streams,url
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock,NaN,NaN,NaN
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock,NaN,NaN,NaN
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock,NaN,NaN,NaN
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock,2021.0,11515.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock,2021.0,267014286.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock,NaN,NaN,NaN
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock,NaN,NaN,NaN
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country,2021.0,105966.0,https://open.spotify.com/track/4w3tQBXhn5345eU...
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop,NaN,NaN,NaN


In [69]:
merged_df_2 = pd.merge(final_top_80s,sum_streaming_2020, on=['song', 'artist'], how='left')
merged_df_2 = merged_df_2.sort_values(by='Hot100 Rank')
merged_df_2

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre,year,streams,url
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock,NaN,NaN,NaN
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock,2020.0,0.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock,NaN,NaN,NaN
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock,2020.0,0.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock,2020.0,65291759.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock,NaN,NaN,NaN
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock,NaN,NaN,NaN
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country,2020.0,163424.0,https://open.spotify.com/track/4w3tQBXhn5345eU...
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop,NaN,NaN,NaN


In [70]:
merged_df_3 = pd.merge(final_top_80s,sum_streaming_2019, on=['song', 'artist'], how='left')
merged_df_3 = merged_df_3.sort_values(by='Hot100 Rank')
merged_df_3

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre,year,streams,url
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock,NaN,NaN,NaN
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock,NaN,NaN,NaN
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock,NaN,NaN,NaN
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock,2019.0,92970.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock,2019.0,56106645.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock,NaN,NaN,NaN
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock,NaN,NaN,NaN
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country,2019.0,1497929.0,https://open.spotify.com/track/4w3tQBXhn5345eU...
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop,NaN,NaN,NaN


In [71]:
merged_df_4 = pd.merge(final_top_80s,sum_streaming_2018, on=['song', 'artist'], how='left')
merged_df_4 = merged_df_4.sort_values(by='Hot100 Rank')
merged_df_4

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre,year,streams,url
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock,NaN,NaN,NaN
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock,2018.0,48560.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock,NaN,NaN,NaN
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock,2018.0,116877.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock,NaN,NaN,NaN
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock,NaN,NaN,NaN
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country,2018.0,558409.0,https://open.spotify.com/track/4w3tQBXhn5345eU...
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop,NaN,NaN,NaN


In [72]:
merged_df_5 = pd.merge(final_top_80s,sum_streaming_2017, on=['song', 'artist'], how='left')
merged_df_5 = merged_df_4.sort_values(by='Hot100 Rank')
merged_df_5

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre,year,streams,url
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock,NaN,NaN,NaN
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock,2018.0,48560.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock,NaN,NaN,NaN
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock,2018.0,116877.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock,NaN,NaN,NaN
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock,NaN,NaN,NaN
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country,2018.0,558409.0,https://open.spotify.com/track/4w3tQBXhn5345eU...
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop,NaN,NaN,NaN


In [74]:
final_top_100_80s = pd.concat([merged_df, merged_df_2, merged_df_3, merged_df_4, merged_df_5], axis=0)
final_top_100_80s = final_top_100_80s.sort_values(by='streams', ascending=False)
final_top_100_80s = final_top_100_80s.drop_duplicates(subset=['song', 'artist'], keep='first')
final_top_100_80s = final_top_100_80s.sort_values(by='Hot100 Rank')
final_top_100_80s.fillna(0, inplace=True)
print(len(final_top_100_80s))
final_top_100_80s.drop(columns=['year', 'url'], inplace=True)
final_top_100_80s

100


,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,duration_ms,time_signature,chorus_hit,sections,country,song_id,genre,streams
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,...,0.904,123.815,224773,4,46.39820,9,United States,3hVrI7snZMTmHCnbwME47u,rock,0.0
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,...,0.596,116.624,228000,4,42.02573,9,United States,0odIT9B9BvOCnXfS0e4lB5,rock,48560.0
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,...,0.237,93.633,266267,4,38.48246,11,United States,0FGxAEMIE5GhyFqFMHRWaO,rock,0.0
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,...,0.548,108.873,245640,4,26.24707,12,United States,2KH16WveTQWT6KOG9Rg6e2,rock,116877.0
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,...,0.740,117.401,253920,4,69.63651,8,United Kingdom,1JSTJqkT5qHq8MDJnJbRE1,rock,267014286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,...,0.267,130.465,211973,4,50.90012,9,United States,5z3ZDMP02xF33yCvPFnct3,rock,0.0
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,...,0.963,149.206,234653,4,18.89341,13,United States,3OeUlriM0EZHdWleJtjoVr,rock,0.0
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,...,0.813,105.390,162467,4,25.12092,7,United States,4w3tQBXhn5345eUXDGBWZG,country,1497929.0
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,...,0.889,176.737,202236,4,19.08299,10,German-speaking Europe,0DfG1ltJnZyq4Tx3ZLL7ZU,pop,0.0


In [75]:
final_top_100_80s.to_csv('final_top_100_80s.csv', index=False)

In [76]:
cursor.execute("SELECT * FROM top_90s")
top_90s = cursor.fetchall()
top_90s = pd.DataFrame(top_90s, columns=cursor.column_names)
top_90s.drop(columns=['target'], inplace=True)
top_90s['Song'] = top_90s['Song'].str.replace('\"', '')
top_90s['Song'] = top_90s['Song'].str.upper()
top_90s = top_90s.rename(columns={'Song': 'song'})
top_90s.rename(columns={'Artist Names': 'artist'}, inplace=True)
top_90s['artist'] = top_90s['artist'].str.replace('\"', '')
top_90s['artist'] = top_90s['artist'].str.upper()
top_90s['song_id'] = top_90s['uri'].str.split(':').str[-1]
top_90s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,song_id
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,0.160000,0.000005,0.2950,0.961,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,0.128000,0.000000,0.0822,0.258,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,0.422000,0.000044,0.0422,0.965,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,0.413000,0.000000,0.1160,0.112,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,0.306000,0.006590,0.1120,0.622,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,0.486000,0.000010,0.1120,0.792,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,0.000501,0.004030,0.3330,0.481,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,0.484000,0.000000,0.0482,0.388,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,0.452000,0.000000,0.1310,0.256,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh


In [77]:
top_90s['genres'] = None
for index, row in top_90s.iterrows():
    song_id = row['song_id']
    track = get_track(song_id)
    artist_ids = [artist['id'] for artist in track['artists']]
    genres = []
    for artist_id in artist_ids:
        artist_info = get_artist(artist_id)
        genres.extend(artist_info.get('genres', []))
        genres = list(set(genres))
    top_90s.at[index, 'genres'] = genres
top_90s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genres
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,0.000005,0.2950,0.961,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,"[mexican classic rock, neo mellow, blues rock,..."
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,0.000000,0.0822,0.258,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,"[country dawn, country road, contemporary coun..."
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,0.000044,0.0422,0.965,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,[tropical]
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,0.000000,0.1160,0.112,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,"[urban contemporary, r&b, contemporary r&b, da..."
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,0.006590,0.1120,0.622,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,"[ectofolk, singer-songwriter, lilith, permanen..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,0.000010,0.1120,0.792,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,"[latin pop, salsa, modern salsa, tropical]"
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,0.004030,0.3330,0.481,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,"[hip pop, hip hop, miami bass, gangster rap, o..."
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,0.000000,0.0482,0.388,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,"[urban contemporary, r&b, contemporary r&b, ne..."
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,0.000000,0.1310,0.256,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,"[quiet storm, r&b, contemporary r&b, urban con..."


In [103]:
top_90s['genre'] = top_90s['genres'].apply(combine_genre)
top_90s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genres,genre
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,0.2950,0.961,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,"[mexican classic rock, neo mellow, blues rock,...",rock
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,0.0822,0.258,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,"[country dawn, country road, contemporary coun...",country
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,0.0422,0.965,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,[tropical],pop
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,0.1160,0.112,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,"[urban contemporary, r&b, contemporary r&b, da...",pop
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,0.1120,0.622,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,"[ectofolk, singer-songwriter, lilith, permanen...",rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,0.1120,0.792,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,"[latin pop, salsa, modern salsa, tropical]",pop
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,0.3330,0.481,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,"[hip pop, hip hop, miami bass, gangster rap, o...",hip hop
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,0.0482,0.388,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,"[urban contemporary, r&b, contemporary r&b, ne...",r&b
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,0.1310,0.256,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,"[quiet storm, r&b, contemporary r&b, urban con...",r&b


In [105]:
final_top_90s = top_90s
final_top_90s.drop(columns=['genres'], inplace=True)
final_top_90s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,0.000005,0.2950,0.961,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,0.000000,0.0822,0.258,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,0.000044,0.0422,0.965,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,0.000000,0.1160,0.112,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,0.006590,0.1120,0.622,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,0.000010,0.1120,0.792,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,0.004030,0.3330,0.481,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,0.000000,0.0482,0.388,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,0.000000,0.1310,0.256,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b


In [106]:
merged_90 = pd.merge(final_top_90s, sum_streaming_2021, on=['song', 'artist'], how='left')
merged_90 = merged_90.sort_values(by='Hot100 Rank')
merged_90

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre,year,streams,url
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock,NaN,NaN,NaN
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country,2021.0,27593.0,https://open.spotify.com/track/7BD50ATrF3Vab5F...
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop,NaN,NaN,NaN
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop,NaN,NaN,NaN
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop,NaN,NaN,NaN
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop,NaN,NaN,NaN
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b,NaN,NaN,NaN
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b,NaN,NaN,NaN


In [107]:
merged_90_2 = pd.merge(final_top_90s, sum_streaming_2020, on=['song', 'artist'], how='left')
merged_90_2 = merged_90_2.sort_values(by='Hot100 Rank')
merged_90_2

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre,year,streams,url
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock,NaN,NaN,NaN
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country,NaN,NaN,NaN
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop,NaN,NaN,NaN
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop,NaN,NaN,NaN
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop,NaN,NaN,NaN
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop,NaN,NaN,NaN
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b,NaN,NaN,NaN
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b,2020.0,81231.0,https://open.spotify.com/track/6mwA6YiKDjAUG8k...


In [108]:
merged_90_3 = pd.merge(final_top_90s, sum_streaming_2019, on=['song', 'artist'], how='left')
merged_90_3 = merged_90_3.sort_values(by='Hot100 Rank')
merged_90_3

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre,year,streams,url
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock,NaN,NaN,NaN
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country,2019.0,0.0,https://open.spotify.com/track/6lo9DfMREEeZaWW...
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop,NaN,NaN,NaN
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop,NaN,NaN,NaN
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop,NaN,NaN,NaN
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop,NaN,NaN,NaN
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b,NaN,NaN,NaN
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b,2019.0,2248386.0,https://open.spotify.com/track/6mwA6YiKDjAUG8k...


In [109]:
merged_90_4 = pd.merge(final_top_90s, sum_streaming_2018, on=['song', 'artist'], how='left')
merged_90_4 = merged_90_4.sort_values(by='Hot100 Rank')
merged_90_4

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre,year,streams,url
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock,NaN,NaN,NaN
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country,NaN,NaN,NaN
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop,NaN,NaN,NaN
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop,NaN,NaN,NaN
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop,NaN,NaN,NaN
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop,NaN,NaN,NaN
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b,NaN,NaN,NaN
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b,2018.0,133250.0,https://open.spotify.com/track/0WWKJsCjOAhymkd...


In [110]:
merged_90_5 = pd.merge(final_top_90s, sum_streaming_2017, on=['song', 'artist'], how='left')
merged_90_5 = merged_90_5.sort_values(by='Hot100 Rank')
merged_90_5

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre,year,streams,url
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock,NaN,NaN,NaN
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country,NaN,NaN,NaN
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop,NaN,NaN,NaN
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop,NaN,NaN,NaN
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop,NaN,NaN,NaN
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop,NaN,NaN,NaN
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b,NaN,NaN,NaN
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b,2017.0,0.0,https://open.spotify.com/track/0WWKJsCjOAhymkd...


In [115]:
final_top_100_90s = pd.concat([merged_90, merged_90_2, merged_90_3, merged_90_4, merged_90_5], axis=0)
final_top_100_90s = final_top_100_90s.sort_values(by='streams', ascending=False)
final_top_100_90s = final_top_100_90s.drop_duplicates(subset=['song', 'artist'], keep='first')
final_top_100_90s = final_top_100_90s.sort_values(by='Hot100 Rank')
final_top_100_90s.fillna(0, inplace=True)
print(len(final_top_100_90s))
final_top_100_90s.drop(columns=['year', 'url'], inplace=True)
final_top_100_90s

99


,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,song_id,genre,streams
0,SMOOTH,SANTANA FEATURING ROB THOMAS,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,...,0.2950,0.961,115.996,294987,4,44.23530,11,0n2SEXB2qoRQg171q7XqeW,rock,0.0
1,HOW DO I LIVE,LEANN RIMES,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,...,0.0822,0.258,128.303,266973,4,43.73251,11,7BD50ATrF3Vab5FQy7vtK8,country,27593.0
2,MACARENA (BAYSIDE BOYS MIX),LOS DEL RIO,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,...,0.0422,0.965,103.018,249382,4,25.74047,15,4oaj36KzXRgDg4McgcTsZK,pop,0.0
3,UN-BREAK MY HEART,TONI BRAXTON,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,...,0.1160,0.112,110.018,270333,4,53.40658,9,5Ihd9HrPvOADyVoonH9ZjB,pop,0.0
4,DOWN SO LONG,JEWEL,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,...,0.1120,0.622,78.501,255147,4,36.83087,10,5WmOYQAMavY2nk7tktsPbZ,rock,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I NEED TO KNOW,MARC ANTHONY,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,...,0.1120,0.792,115.061,227707,4,31.46063,10,7ffwRz8lZyDOE4Vj58Lo72,pop,0.0
96,JUMP,KRIS KROSS,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,...,0.3330,0.481,101.972,195107,4,47.80584,10,27AHAtAirQapVldIm4c9ZX,hip hop,0.0
97,NOBODY,KEITH SWEAT FEATURING ATHENA CAGE,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,...,0.0482,0.388,117.907,264773,4,32.47790,12,4SF1747p541umnykBp352Q,r&b,0.0
98,BACK AT ONE,BRIAN MCKNIGHT,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,...,0.1310,0.256,129.752,263667,4,40.51017,7,6mwA6YiKDjAUG8kWvRRUPh,r&b,2248386.0


In [116]:
final_top_100_90s.to_csv('final_top_100_90s.csv', index=False)